In [8]:
import pandas as pd
import os 
import zipfile


with zipfile.ZipFile('homework.zip', 'r') as zip_ref:
    files = zip_ref.namelist()
    
    for file in files:
        if file.endswith('.csv'):
            zip_ref.extract(file)
            df = pd.read_csv(file)
            print(df.head())
            
            
def calculate_time_domain_features(df):
    time_domain_features = {}
    for axis in ['X', 'Y', 'Z']:
        axis_data = df[f'accelerometer_{axis}']
        time_domain_features[f'Mean_{axis}'] = axis_data.mean()
        time_domain_features[f'Std_{axis}'] = axis_data.std()
        time_domain_features[f'Max_{axis}'] = axis_data.max()
        time_domain_features[f'Min_{axis}'] = axis_data.min()
    return time_domain_features

   accelerometer_X  accelerometer_Y  accelerometer_Z
0         1.000776         4.616021         8.576031
1         0.718261         4.209007         8.446744
2        -0.909797        -0.282516         9.203311
3         5.099650         0.148441         8.418014
4         1.762132        -0.162806         9.251195
   accelerometer_X  accelerometer_Y  accelerometer_Z
0        -0.100556        -0.124498         9.763555
1        -0.081403        -0.110133         9.744401
2        -0.124498        -0.153229         9.773131
3        -0.320823        -0.191536         9.792285
4         0.148441        -0.234632         9.768343
   accelerometer_X  accelerometer_Y  accelerometer_Z
0        -0.095768         0.234632         9.763555
1        -0.086191         0.234632         9.753978
2        -0.105345         0.225055         9.753978
3        -0.086191         0.234632         9.777920
4        -0.076614         0.225055         9.744401
   accelerometer_X  accelerometer_Y  accelerom